In [1]:
import sqlite3 as sql

import numpy as np
import pandas as pd
from pandas import cut, read_sql
import pickle as pkl
from random import choices
from sklearn import metrics
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy.optimize as optimize

In [2]:
input_csv_cascade_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_MP_lvl3/inference/Troels_cascade_events/clean_cascade_events_MC.csv"
input_csv_cascade_RD = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_MP_lvl3/inference/Troels_cascade_events/clean_cascade_events_RD.csv"
outdir = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_MP_lvl3/inference/Troels_cascade_events"
csv_cascade_MC = pd.read_csv(input_csv_cascade_MC).sort_values('event_no').reset_index(drop = True)
csv_cascade_RD = pd.read_csv(input_csv_cascade_RD).sort_values('event_no').reset_index(drop = True)

In [9]:
print(csv_cascade_MC.columns)
print(csv_cascade_RD.columns)
csv_cascade_MC.drop(['Unnamed: 0'],axis=1,inplace=True)
csv_cascade_RD.drop(['Unnamed: 0'],axis=1,inplace=True)


Index(['Unnamed: 0', 'event_no', 'track_mu_pred', 'track_mu', 'pid_noise_pred',
       'pid_muon_pred', 'pid_neutrino_pred', 'pid', 'pid_neutrino_pred_logit',
       'energy_pred', 'energy', 'azimuth_pred', 'azimuth_kappa', 'azimuth',
       'zenith_pred', 'zenith_kappa', 'zenith'],
      dtype='object')
Index(['Unnamed: 0', 'event_no', 'track_mu_pred', 'pid_noise_pred',
       'pid_muon_pred', 'pid_neutrino_pred', 'pid_neutrino_pred_logit',
       'energy_pred', 'azimuth_pred', 'azimuth_kappa', 'zenith_pred',
       'zenith_kappa'],
      dtype='object')


In [10]:
print(csv_cascade_MC.columns)
print(csv_cascade_RD.columns)

Index(['event_no', 'track_mu_pred', 'track_mu', 'pid_noise_pred',
       'pid_muon_pred', 'pid_neutrino_pred', 'pid', 'pid_neutrino_pred_logit',
       'energy_pred', 'energy', 'azimuth_pred', 'azimuth_kappa', 'azimuth',
       'zenith_pred', 'zenith_kappa', 'zenith'],
      dtype='object')
Index(['event_no', 'track_mu_pred', 'pid_noise_pred', 'pid_muon_pred',
       'pid_neutrino_pred', 'pid_neutrino_pred_logit', 'energy_pred',
       'azimuth_pred', 'azimuth_kappa', 'zenith_pred', 'zenith_kappa'],
      dtype='object')


In [12]:
indir_db_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_database_Peter_and_Morten/merged_database/osc_next_level3_v2.00_genie_muongun_noise_120000_140000_160000_130000_888003.db"
Npulses_MC = []
Ndoms_MC = []
Nstrings_MC = []
event_no_MC = []
event_times_MC = []

#Load in truth data
for i in range(100):
    with sql.connect(indir_db_MC) as con:
        query = f"""
        SELECT
            event_no, event_time, dom_x, dom_y, dom_z
        FROM 
            SplitInIcePulses
        WHERE
            event_no = {csv_cascade_MC['event_no'][i]}
        """
        MC_extra_data = read_sql(query,con)
    event_no_MC.append(csv_cascade_MC['event_no'][i])
    Npulses_MC.append(len(MC_extra_data))
    Ndoms_MC.append(MC_extra_data.groupby(['dom_x', 'dom_y','dom_z']).ngroups)
    Nstrings_MC.append(MC_extra_data.groupby(['dom_x', 'dom_y']).ngroups)
    event_times_MC.append(MC_extra_data['event_time'][0])

In [17]:
# print(Npulses_MC)
# print(Ndoms_MC)
# print(Nstrings_MC)
# print(event_no_MC)
# print(event_times_MC)

: 

In [14]:
indir_db_RD = "/groups/icecube/petersen/GraphNetDatabaseRepository/dev_lvl3_genie_burnsample/dev_lvl3_genie_burnsample_v5.db"
Npulses_RD = []
Ndoms_RD = []
Nstrings_RD = []
event_no_RD = []
first_dom_time_RD = []
for i in range(100):
    #Load in truth data
    with sql.connect(indir_db_RD) as con:
        query = f"""
        SELECT
            event_no, dom_time, dom_x, dom_y, dom_z
        FROM 
            SplitInIcePulses
        WHERE
            event_no = {csv_cascade_RD['event_no'][i]}
        """
        RD_extra_data = read_sql(query,con)
    event_no_RD.append(csv_cascade_RD['event_no'][i])
    Npulses_RD.append(len(RD_extra_data))
    Ndoms_RD.append(RD_extra_data.groupby(['dom_x', 'dom_y','dom_z']).ngroups)
    Nstrings_RD.append(RD_extra_data.groupby(['dom_x', 'dom_y']).ngroups)
    first_dom_time_RD.append(RD_extra_data['dom_time'][0])

In [16]:
#print(RD_extra_data)
# print('there are this many pulses: ',len(RD_extra_data))
# print('there are this many unique doms: ',RD_extra_data.groupby(['dom_x', 'dom_y','dom_z']).ngroups)
# print('there are this many unique string: ',RD_extra_data.groupby(['dom_x', 'dom_y']).ngroups)
#print(Npulses_RD)
#print(Ndoms_RD)
#print(Nstrings_RD)
#print(event_no_RD)
#print(first_dom_time_RD)
csv_cascade_RD_final = csv_cascade_RD
csv_cascade_RD_final['N_pulses'] = Npulses_RD
csv_cascade_RD_final['N_string'] = Nstrings_RD
csv_cascade_RD_final['N_doms'] = Ndoms_RD
csv_cascade_RD_final['First_dom_time'] = first_dom_time_RD

csv_cascade_MC_final = csv_cascade_MC
csv_cascade_MC_final['N_pulses'] = Npulses_MC
csv_cascade_MC_final['N_string'] = Nstrings_MC
csv_cascade_MC_final['N_doms'] = Ndoms_MC
csv_cascade_MC_final['First_dom_time'] = event_times_MC

ValueError: Length of values (100) does not match length of index (245049)

In [ ]:
csv_cascade_RD_final.to_csv(outdir + 'clean_cascade_events_RD_final',index=False)
csv_cascade_MC_final.to_csv(outdir + 'clean_cascade_events_MC_final',index=False)